Necessary (external) libraries:
- jiwer
- python-docx


In [25]:
import jiwer
#import batchalign
import os
import pathlib
from docx import Document
import re
import pandas as pd

from asr_intro_handler import preprocess_transcript_pair
from audio_quality_metrics import get_audio_metrics_dataframe

#pd.set_option("display.max_columns", 50) 
#pd.set_option("display.max_rows", 50)


# Data Preprocessing

In [26]:
# HOW TO HANDLE INTRO 
# Choose how to handle introductions
# Options: 
# "noop" - ignores intro handling
# "replace" - replaces placeholder in Human transcript with Batchaling2 generated intro
# "remove" - deletes intro from Batchaling2
mode = "replace"

## Setting up the Paths

In [27]:
#setting up the paths

path = pathlib.Path().resolve()

#path to the directory with the Batchalign transcriptions
hypothesis_path = path / "transkrypty_czyszczonych_nagrań"

#path to the directory with the human transcriptions
reference_path = path / "czyszczone_transkrypcje"

#path to the directory where the copy of the human transcriptions is saved
reference_path_copy = path / "czyszczone_transkrypcje_copy"

#path to the directory where the copy of the Batchalign transcriptions is saved
hypothesis_path_copy = path / "transkrypty_czyszczonych_nagrań_copy"

used_audio_path = path / "czyszczone_nagrania"  # Audio files

#### Script for reseting the files inside the reference_path

In [ ]:
# #ONLY RUN THIS SCRIPT IF YOU WANT TO RESET THE REFERENCE PATH

# # to unncomment the code, select it and press Ctrl + / (or Cmd + / on Mac) 
# # This script will delete all files in the reference_path and copy files from reference_path_copy to reference_path

# import shutil

# # 1. Delete all files from reference_path
# for file in reference_path.iterdir():
#     if file.is_file():
#         file.unlink()

# # 2. Copy all files from reference_path_copy to reference_path
# for file in reference_path_copy.iterdir():
#     if file.is_file():
#         target = reference_path / file.name
#         shutil.copy2(file, target)

#### Script for reseting the files inside the hypothesis_path

In [ ]:
# #ONLY RUN THIS SCRIPT IF YOU WANT TO 'RESET' THE HYPOTHESIS PATH

# # to unncomment the code, select it and press Ctrl + / (or Cmd + / on Mac) 
# # This script will delete all files in the hypothesis_path and copy files from hypothesis_path_copy to hypothesis_path

# import shutil

# # 1. Delete all files from hypothesis_path
# for file in hypothesis_path.iterdir():
#     if file.is_file():
#         file.unlink()

# # 2. Copy all files from hypothesis_path_copy to hypothesis_path
# for file in hypothesis_path_copy.iterdir():
#     if file.is_file():
#         target = hypothesis_path / file.name
#         shutil.copy2(file, target)

In [30]:
def remove_speaker_labels(text: str) -> str:
    """
    Remove speaker labels from transcripts.
    
    Removes patterns like:
    - *EXP:, *CHI:, *PAR0:, *PAR1:, *PAR2:, etc.
    - Any *WORD: pattern at the start of lines or after newlines
    
    Args:
        text: Input transcript with speaker labels
    
    Returns:
        Text with speaker labels removed
    """
    # Remove speaker labels: *WORD: or *WORDdigit:
    # Pattern matches: *EXP:, *CHI:, *PAR0:, *PAR1:, etc.
    cleaned = re.sub(r'\*[A-Za-z]+\d*:\s*', '', text)
    
    return cleaned

## Preprocessing the reference_path

### Converting the reference files to .txt

In [31]:
#converting all files in the reference_path to a .txt format

def convert_docx_or_cha_to_txt(file_path):
    """
    
    !!!!!
    Remember to have the backup of the original files before running this function, as it will delete the original files after conversion
    !!!!! 

    Converts .docx or .cha files to .txt format and deletes the original file.

    """
    if file_path.suffix.lower() == '.docx':
        new_file_path = file_path.with_suffix('.txt')
        try:
            doc = Document(file_path)
            content = "\n".join([para.text for para in doc.paragraphs])
            with open(new_file_path, 'w', encoding='utf-8') as f:
                f.write(content)
            os.remove(file_path)
            print(f"Converted and removed: {file_path}")
            return new_file_path
        except Exception as e:
            print(f"Error converting {file_path}: {e}")
            return file_path
        
    elif file_path.suffix.lower() == '.cha':
        new_file_path = file_path.with_suffix('.txt')
        try:
            with open(file_path, 'r', encoding='utf-8') as f_in:
                content = f_in.read()
            with open(new_file_path, 'w', encoding='utf-8') as f_out:
                f_out.write(content)
            os.remove(file_path)
            print(f"Converted and removed: {file_path}")
            return new_file_path
        except Exception as e:
            print(f"Error converting {file_path}: {e}")
            return file_path
    return file_path

In [32]:
# Run the function for all .docx and .cha files in the reference_path
for file in os.listdir(reference_path):
    file_path = reference_path / file
    if file_path.is_file() and file_path.suffix.lower() in ['.docx', '.cha']:
        convert_docx_or_cha_to_txt(file_path)

Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bm225_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf035_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf025_NAp_transkrypcja_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf224_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bm135_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Mf115_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf023_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bm022_NAp_transkrypcja_clean.docx
Converted and removed: /Users/kajtek/Desk

### Cleaning the reference files

In [33]:
# Cleaning the files in the reference_path

def clean_transcript_file(file_path):
    """
    Cleans transcript files by:
    - Removing all lines before the first *CHI: or *EXP:
    - Removing all lines starting with %com: anywhere in the file
    - Removing trailing empty lines and lines like '@End' or similar markers at the end
    Overwrites the file in place.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Find the index of the first line starting with *CHI: or *EXP:
    start_idx = next(
        (i for i, line in enumerate(lines) if line.lstrip().startswith("*CHI:") or line.lstrip().startswith("*EXP:")),
        None
    )

    if start_idx is not None:
        # Remove %com: lines after trimming the start
        cleaned_lines = [
            line for line in lines[start_idx:]
            if not line.lstrip().lower().startswith("%com:")
        ]
        # Remove trailing empty lines and lines like '@End'
        while cleaned_lines and (cleaned_lines[-1].strip() == "" or cleaned_lines[-1].strip().lower() == "@end"):
            cleaned_lines.pop()
        with open(file_path, 'w', encoding='utf-8') as f:
            f.writelines(cleaned_lines)
        print(f"Cleaned: {file_path}")
    else:
        print(f"No *CHI: or *EXP: found in {file_path}")

In [34]:
# Running the cleaning function for all .txt files in the reference_path
reference_path = pathlib.Path().resolve() / "czyszczone_transkrypcje"
for file in os.listdir(reference_path):
    file_path = reference_path / file
    if file_path.is_file() and file_path.suffix.lower() == '.txt':
        clean_transcript_file(file_path)

Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf064_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Mm022_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Mf110_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bm020_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf068_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bm015_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Bf127_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Mm017_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszczone_transkrypcje/Mm800_NAp_transkrypcja_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/czyszcz

## Preprocessing the hypothesis_path

### Converting the hypothesis files to .txt

In [35]:
# converting all files in the hypothesis_path to a .txt format

def convert_cha_to_txt(file_path):
    """
    
    !!!!!
    Remember to have the backup of the original files before running this function, as it will delete the original files after conversion
    !!!!!

    Converts .cha files to .txt format and deletes the original file.


    """
    if file_path.suffix.lower() == '.cha':
        new_file_path = file_path.with_suffix('.txt')
        try:
            with open(file_path, 'r', encoding='utf-8') as f_in:
                content = f_in.read()
            with open(new_file_path, 'w', encoding='utf-8') as f_out:
                f_out.write(content)
            os.remove(file_path)
            print(f"Converted and removed: {file_path}")
            return new_file_path
        except Exception as e:
            print(f"Error converting {file_path}: {e}")
            return file_path
    return file_path

In [36]:
# Run the function for all .cha files in the hypothesis_path
for file in os.listdir(hypothesis_path):
    file_path = hypothesis_path / file
    if file_path.is_file() and file_path.suffix.lower() == '.cha':
        convert_cha_to_txt(file_path)

Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm136_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf113_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm062_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf117_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bf0148_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf099_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf033_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm016_NAp_clean.cha
Converted and removed: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_

### Cleaning the hypothesis files

In [37]:
def clean_hypothesis_file(file_path):
    """
    Cleans hypothesis files by:
    - Removing all lines before the first line starting with *PAR (e.g., *PAR0:, *PAR1:, etc.)
    - Removing all timestamp patterns ->  XXXX_XXXX
    - Removing the last line if it is '@End' (case-insensitive) and any trailing empty lines
    Overwrites the file in place.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Find the index of the first line starting with *PAR
    start_idx = next(
        (i for i, line in enumerate(lines) if line.lstrip().startswith("*PAR")),
        None
    )

    if start_idx is not None:
        cleaned_lines = lines[start_idx:]
        # Remove timestamp patterns
        cleaned_lines = [re.sub(r"\x15\d+_\d+\x15", "", line) for line in cleaned_lines]
        
        
        # Remove trailing lines that are empty or '@End'
        while cleaned_lines and (cleaned_lines[-1].strip() == "" or cleaned_lines[-1].strip().lower() == "@end"):
            cleaned_lines.pop()
        with open(file_path, 'w', encoding='utf-8') as f:
            f.writelines(cleaned_lines)
        print(f"Cleaned: {file_path}")
    else:
        print(f"No *PAR found in {file_path}")

In [38]:
# Running the cleaning function for all .txt files in the hypothesis_path
hypothesis_path = pathlib.Path().resolve() / "transkrypty_czyszczonych_nagrań"
for file in os.listdir(hypothesis_path):
    file_path = hypothesis_path / file
    if file_path.is_file() and file_path.suffix.lower() == '.txt':
        clean_hypothesis_file(file_path)

Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bf135_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf009_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm015_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bf076_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm028_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mm114_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf034_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bf249_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Mf110_NAp_clean.txt
Cleaned: /Users/kajtek/Desktop/Praktyki/analizy/transkrypty_czyszczonych_nagrań/Bm135_NAp_clean.txt


# Data analysis

## Data transformation

In [39]:
#transformation pipeline

transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(), #expands common English contractions like "don't" to "do not"
        jiwer.RemoveEmptyStrings(), #removes empty strings
        jiwer.ToLowerCase(), #converts all characters to lowercase
        #jiwer.RemoveMultipleSpaces(), #removes multiple spaces between words
        jiwer.SubstituteRegexes({r"\s+": " "}), # replaces all multipiles, \n, \t, etc with a single space 
        jiwer.Strip(), #removes leading and trailing spaces
        jiwer.RemovePunctuation(), #removes punctuation marks (e.g., ., !, ?, etc.)
        #jiwer.RemoveKaldiNonWords(), #removes Kaldi non-words (e.g., <unk>, <s>, </s>, etc.)
        jiwer.ReduceToListOfListOfWords(), #reduces the transcription to a list of lists of words
    ]
)

In [40]:
# List all .txt files in both folders
ref_files = [f for f in os.listdir(reference_path) if f.endswith('.txt')]
hyp_files = [f for f in os.listdir(hypothesis_path) if f.endswith('.txt')]

def extract_key(filename):
    # Match 2 letters, 3 or 4 digits, underscore, NAp (e.g., Ab123_NAp or Ab1234_NAp)
    match = re.match(r"^[A-Za-z]{2}\d{3,4}_NAp", filename)
    if match:
        return match.group(0)
    return filename  # fallback: whole filename if not matched

df_ref = pd.DataFrame({
    'key': [extract_key(f) for f in ref_files],
    'ref_file': ref_files
})
df_hyp = pd.DataFrame({
    'key': [extract_key(f) for f in hyp_files],
    'hyp_file': hyp_files
})

# Merge to get only matching pairs
df = pd.merge(df_ref, df_hyp, on='key')

display(df)

,key,ref_file,hyp_file
0,Mf110_NAp,Mf110_NAp_transkrypcja_clean.txt,Mf110_NAp_clean.txt
1,Bm020_NAp,Bm020_NAp_transkrypcja_clean.txt,Bm020_NAp_clean.txt
2,Bf068_NAp,Bf068_NAp_transkrypcja_clean.txt,Bf068_NAp_clean.txt
3,Bm015_NAp,Bm015_NAp_transkrypcja_clean.txt,Bm015_NAp_clean.txt
4,Mm800_NAp,Mm800_NAp_transkrypcja_clean.txt,Mm800_NAp_clean.txt
...,...,...,...
88,Bm010_NAp,Bm010_NAp_transkrypcja_clean.txt,Bm010_NAp_clean.txt
89,Mf119_NAp,Mf119_NAp_transkrypcja_clean.txt,Mf119_NAp_clean.txt
90,Bm025_NAp,Bm025_NAp_transkrypcja_clean.txt,Bm025_NAp_clean.txt
91,Mf115_NAp,Mf115_NAp_transkrypcja_clean.txt,Mf115_NAp_clean.txt


In [41]:
#adding string columns to the DataFrame
# Load raw strings
df['ref_string_raw'] = df['ref_file'].apply(lambda x: (reference_path / x).read_text(encoding='utf-8'))
df['hyp_string_raw'] = df['hyp_file'].apply(lambda x: (hypothesis_path / x).read_text(encoding='utf-8'))

# Remove speaker labels (on raw input)
df['ref_string_clean'] = df['ref_string_raw'].apply(remove_speaker_labels)
df['hyp_string_clean'] = df['hyp_string_raw'].apply(remove_speaker_labels)

# Apply intro-handling
processed_refs = []
processed_hyps = []
has_placeholder_list = []
placeholder_text_list = []
intro_extracted_list = []
intro_length_list = []
action_taken_list = []

for ref, hyp in zip(df['ref_string_clean'], df['hyp_string_clean']):
    pr, ph, meta = preprocess_transcript_pair(ref, hyp, method=mode)
    processed_refs.append(pr)
    processed_hyps.append(ph)
    has_placeholder_list.append(meta['has_placeholder'])
    placeholder_text_list.append(" | ".join(meta['placeholders']))
    intro_extracted_list.append(int(meta['intro_extracted']))
    intro_length_list.append(len(meta['intro_text']))
    action_taken_list.append(meta['action_taken'])

df['ref_string'] = processed_refs
df['hyp_string'] = processed_hyps
df['has_placeholder'] = has_placeholder_list
df['placeholder_text'] = placeholder_text_list
df['intro_extracted'] = intro_extracted_list
df['intro_length_chars'] = intro_length_list
df['intro_action'] = action_taken_list


In [42]:
df

,key,ref_file,hyp_file,ref_string_raw,hyp_string_raw,ref_string_clean,hyp_string_clean,ref_string,hyp_string,has_placeholder,placeholder_text,intro_extracted,intro_length_chars,intro_action
0,Mf110_NAp,Mf110_NAp_transkrypcja_clean.txt,Mf110_NAp_clean.txt,"*CHI:\tTutaj zielony razem, no bo by się nie z...","*PAR0:\tTutaj zieloną razem , no bo by się nie...","Tutaj zielony razem, no bo by się nie zmieścił...","Tutaj zieloną razem , no bo by się nie zmieści...","Tutaj zielony razem, no bo by się nie zmieścił...","Tutaj zieloną razem , no bo by się nie zmieści...",0,,1,1371,no_placeholder_no_action
1,Bm020_NAp,Bm020_NAp_transkrypcja_clean.txt,Bm020_NAp_clean.txt,*CHI:\tMhm.\n*CHI:\tPowiedziałaś że\n*EXP:\tOk...,*PAR1:\tTy . \n*PAR0:\tJesteś Michał i Twój . ...,"Mhm.\nPowiedziałaś że\nOkay, Ty jesteś Michał ...",Ty . \nJesteś Michał i Twój . \nKod to BM020 ....,"Mhm.\nPowiedziałaś że\nOkay, Ty jesteś Michał ...",Ty . \nJesteś Michał i Twój . \nKod to BM020 ....,0,,1,419,no_placeholder_no_action
2,Bf068_NAp,Bf068_NAp_transkrypcja_clean.txt,Bf068_NAp_clean.txt,*EXP:\tLiliana.\n*EXP:\ttwój numer to be ef ze...,"*PAR1:\tJa nazywam się Lucyna Jachacy , a Ty n...",Liliana.\ntwój numer to be ef zero sześćdziesi...,"Ja nazywam się Lucyna Jachacy , a Ty nazywasz ...",Liliana.\ntwój numer to be ef zero sześćdziesi...,"Ja nazywam się Lucyna Jachacy , a Ty nazywasz ...",0,,1,866,no_placeholder_no_action
3,Bm015_NAp,Bm015_NAp_transkrypcja_clean.txt,Bm015_NAp_clean.txt,*EXP:\tMhm. \n*EXP:\tTo ty masz na imię...\n*C...,*PAR2:\tTrochę . \n*PAR1:\tA wiesz co to jest ...,Mhm. \nTo ty masz na imię...\nWojtek.\nWojtek....,Trochę . \nA wiesz co to jest opowiadanie ? \n...,Mhm. \nTo ty masz na imię...\nWojtek.\nWojtek....,Trochę . \nA wiesz co to jest opowiadanie ? \n...,0,,1,572,no_placeholder_no_action
4,Mm800_NAp,Mm800_NAp_transkrypcja_clean.txt,Mm800_NAp_clean.txt,"*CHI:\tTen…\n*CHI:\tYy, w układanki?\n*EXP:\tY...",*PAR0:\tJak tam powie +... \n*PAR0:\tPoczekaj ...,"Ten…\nYy, w układanki?\nYy…\nCzyli=cyli jak, y...",Jak tam powie +... \nPoczekaj . \nBolesławMM80...,"Ten…\nYy, w układanki?\nYy…\nCzyli=cyli jak, y...",Jak tam powie +... \nPoczekaj . \nBolesławMM80...,0,,1,209,no_placeholder_no_action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Bm010_NAp,Bm010_NAp_transkrypcja_clean.txt,Bm010_NAp_clean.txt,"*EXP:\tPowiedz mi, czy lubisz słuchać bajek i ...","*PAR1:\tTymuś , nasze ostatnie zadanie na dzis...","Powiedz mi, czy lubisz słuchać bajek i różnych...","Tymuś , nasze ostatnie zadanie na dzisiaj . \n...","Powiedz mi, czy lubisz słuchać bajek i różnych...","Tymuś , nasze ostatnie zadanie na dzisiaj . \n...",0,,1,5397,no_placeholder_no_action
89,Mf119_NAp,Mf119_NAp_transkrypcja_clean.txt,Mf119_NAp_clean.txt,*EXP:\tAha...\n*CHI:\tLubię tylko bajki słucha...,*PAR0:\tLubię tylko bajki słuchać . \n*PAR1:\t...,"Aha...\nLubię tylko bajki słuchać.\nOk, a jaki...",Lubię tylko bajki słuchać . \nOkej A jakie baj...,"Aha...\nLubię tylko bajki słuchać.\nOk, a jaki...",Lubię tylko bajki słuchać . \nOkej A jakie baj...,1,[standardowa rozgrzewka],1,1816,replaced_placeholder_with_intro
90,Bm025_NAp,Bm025_NAp_transkrypcja_clean.txt,Bm025_NAp_clean.txt,*EXP:\tOkej.\n*EXP:\tMożemy zacząć.\n*EXP:\tTo...,*PAR1:\tTo jest numer BN025 i dzisiaj jest dwa...,Okej.\nMożemy zacząć.\nTo jest numer Bm025 i d...,To jest numer BN025 i dzisiaj jest dwadzieścia...,Okej.\nMożemy zacząć.\nTo jest numer Bm025 i d...,To jest numer BN025 i dzisiaj jest dwadzieścia...,0,,1,1231,no_placeholder_no_action
91,Mf115_NAp,Mf115_NAp_transkrypcja_clean.txt,Mf115_NAp_clean.txt,"*EXP:\tZaraz podam twój tajny kod, ja jestem A...",*PAR1:\tTeraz podam twój tajny kod . \n*PAR1:\...,"Zaraz podam twój tajny kod, ja jestem Asia, dz...",Teraz podam twój tajny kod . \nJa jestem Asia ...,"Zaraz podam twój tajny kod, ja jestem Asia, dz...",Teraz podam twój tajny kod . \nJa jestem Asia ...,1,[standardowa rozgrzewka],1,720,replaced_placeholder_with_intro


In [43]:
#applying the transformation pipeline to the strings in the DataFrame
df['ref_transformed'] = df['ref_string'].apply(lambda x: " ".join(sum(transforms(x), [])))
df['hyp_transformed'] = df['hyp_string'].apply(lambda x: " ".join(sum(transforms(x), [])))


In [44]:
# Create directories for transformed files if they don't exist
transformed_ref_path = path / "transformed_files" / "reference"
transformed_hyp_path = path / "transformed_files" / "hypothesis"
transformed_ref_path.mkdir(parents=True, exist_ok=True)
transformed_hyp_path.mkdir(parents=True, exist_ok=True)

# Save transformed reference files
for _, row in df.iterrows():
    ref_output_path = transformed_ref_path / f"{row['key']}_ref_transformed.txt"
    with open(ref_output_path, 'w', encoding='utf-8') as f:
        f.write(row['ref_transformed'])
    
    # Save transformed hypothesis files
    hyp_output_path = transformed_hyp_path / f"{row['key']}_hyp_transformed.txt"
    with open(hyp_output_path, 'w', encoding='utf-8') as f:
        f.write(row['hyp_transformed'])

print(f"Saved {len(df)} transformed reference files to {transformed_ref_path}")
print(f"Saved {len(df)} transformed hypothesis files to {transformed_hyp_path}")

Saved 93 transformed reference files to /Users/kajtek/Desktop/Praktyki/analizy/transformed_files/reference
Saved 93 transformed hypothesis files to /Users/kajtek/Desktop/Praktyki/analizy/transformed_files/hypothesis


In [45]:
df

,key,ref_file,hyp_file,ref_string_raw,hyp_string_raw,ref_string_clean,hyp_string_clean,ref_string,hyp_string,has_placeholder,placeholder_text,intro_extracted,intro_length_chars,intro_action,ref_transformed,hyp_transformed
0,Mf110_NAp,Mf110_NAp_transkrypcja_clean.txt,Mf110_NAp_clean.txt,"*CHI:\tTutaj zielony razem, no bo by się nie z...","*PAR0:\tTutaj zieloną razem , no bo by się nie...","Tutaj zielony razem, no bo by się nie zmieścił...","Tutaj zieloną razem , no bo by się nie zmieści...","Tutaj zielony razem, no bo by się nie zmieścił...","Tutaj zieloną razem , no bo by się nie zmieści...",0,,1,1371,no_placeholder_no_action,tutaj zielony razem no bo by się nie zmieściły...,tutaj zieloną razem no bo by się nie zmieściły...
1,Bm020_NAp,Bm020_NAp_transkrypcja_clean.txt,Bm020_NAp_clean.txt,*CHI:\tMhm.\n*CHI:\tPowiedziałaś że\n*EXP:\tOk...,*PAR1:\tTy . \n*PAR0:\tJesteś Michał i Twój . ...,"Mhm.\nPowiedziałaś że\nOkay, Ty jesteś Michał ...",Ty . \nJesteś Michał i Twój . \nKod to BM020 ....,"Mhm.\nPowiedziałaś że\nOkay, Ty jesteś Michał ...",Ty . \nJesteś Michał i Twój . \nKod to BM020 ....,0,,1,419,no_placeholder_no_action,mhm powiedziałaś że okay ty jesteś michał i tw...,ty jesteś michał i twój kod to bm020 lubisz sł...
2,Bf068_NAp,Bf068_NAp_transkrypcja_clean.txt,Bf068_NAp_clean.txt,*EXP:\tLiliana.\n*EXP:\ttwój numer to be ef ze...,"*PAR1:\tJa nazywam się Lucyna Jachacy , a Ty n...",Liliana.\ntwój numer to be ef zero sześćdziesi...,"Ja nazywam się Lucyna Jachacy , a Ty nazywasz ...",Liliana.\ntwój numer to be ef zero sześćdziesi...,"Ja nazywam się Lucyna Jachacy , a Ty nazywasz ...",0,,1,866,no_placeholder_no_action,liliana twój numer to be ef zero sześćdziesiąt...,ja nazywam się lucyna jachacy a ty nazywasz si...
3,Bm015_NAp,Bm015_NAp_transkrypcja_clean.txt,Bm015_NAp_clean.txt,*EXP:\tMhm. \n*EXP:\tTo ty masz na imię...\n*C...,*PAR2:\tTrochę . \n*PAR1:\tA wiesz co to jest ...,Mhm. \nTo ty masz na imię...\nWojtek.\nWojtek....,Trochę . \nA wiesz co to jest opowiadanie ? \n...,Mhm. \nTo ty masz na imię...\nWojtek.\nWojtek....,Trochę . \nA wiesz co to jest opowiadanie ? \n...,0,,1,572,no_placeholder_no_action,mhm to ty masz na imię wojtek wojtek a dziś ma...,trochę a wiesz co to jest opowiadanie nie znas...
4,Mm800_NAp,Mm800_NAp_transkrypcja_clean.txt,Mm800_NAp_clean.txt,"*CHI:\tTen…\n*CHI:\tYy, w układanki?\n*EXP:\tY...",*PAR0:\tJak tam powie +... \n*PAR0:\tPoczekaj ...,"Ten…\nYy, w układanki?\nYy…\nCzyli=cyli jak, y...",Jak tam powie +... \nPoczekaj . \nBolesławMM80...,"Ten…\nYy, w układanki?\nYy…\nCzyli=cyli jak, y...",Jak tam powie +... \nPoczekaj . \nBolesławMM80...,0,,1,209,no_placeholder_no_action,ten yy w układanki yy czyli=cyli jak yy jak ta...,jak tam powie + poczekaj bolesławmm800 a jak t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Bm010_NAp,Bm010_NAp_transkrypcja_clean.txt,Bm010_NAp_clean.txt,"*EXP:\tPowiedz mi, czy lubisz słuchać bajek i ...","*PAR1:\tTymuś , nasze ostatnie zadanie na dzis...","Powiedz mi, czy lubisz słuchać bajek i różnych...","Tymuś , nasze ostatnie zadanie na dzisiaj . \n...","Powiedz mi, czy lubisz słuchać bajek i różnych...","Tymuś , nasze ostatnie zadanie na dzisiaj . \n...",0,,1,5397,no_placeholder_no_action,powiedz mi czy lubisz słuchać bajek i różnych ...,tymuś nasze ostatnie zadanie na dzisiaj jestem...
89,Mf119_NAp,Mf119_NAp_transkrypcja_clean.txt,Mf119_NAp_clean.txt,*EXP:\tAha...\n*CHI:\tLubię tylko bajki słucha...,*PAR0:\tLubię tylko bajki słuchać . \n*PAR1:\t...,"Aha...\nLubię tylko bajki słuchać.\nOk, a jaki...",Lubię tylko bajki słuchać . \nOkej A jakie baj...,"Aha...\nLubię tylko bajki słuchać.\nOk, a jaki...",Lubię tylko bajki słuchać . \nOkej A jakie baj...,1,[standardowa rozgrzewka],1,1816,replaced_placeholder_with_intro,aha lubię tylko bajki słuchać ok a jakie bajki...,lubię tylko bajki słuchać okej a jakie bajki s...
90,Bm025_NAp,Bm025_NAp_transkrypcja_clean.txt,Bm025_NAp_clean.txt,*EXP:\tOkej.\n*EXP:\tMożemy zacząć.\n*EXP:\tTo...,*PAR1:\tTo jest numer

In [46]:
# Calculate WER and CER for each pair after transformation
results = []
for _, row in df.iterrows():
    # with open(reference_path / row['ref_file'], encoding='utf-8') as f:
    #     ref_text = f.read()
    #     ref_text = remove_speaker_labels(ref_text)
    # with open(hypothesis_path / row['hyp_file'], encoding='utf-8') as f:
    #     hyp_text = f.read()
    #     hyp_text = remove_speaker_labels(hyp_text)

    ref_text = row['ref_string']
    hyp_text = row['hyp_string']
    
    # Apply the transformation pipeline
    ref_transformed = " ".join(sum(transforms(ref_text), []))
    hyp_transformed = " ".join(sum(transforms(hyp_text), []))
    results.append({
        'key': row['key'],
        'ref_file': row['ref_file'],
        'hyp_file': row['hyp_file'],
        'WER': jiwer.wer(ref_transformed, hyp_transformed),
        'CER': jiwer.cer(ref_transformed, hyp_transformed),
        'WIL': jiwer.wil(ref_transformed, hyp_transformed),
        'MER': jiwer.mer(ref_transformed, hyp_transformed),
        'WIP': jiwer.wip(ref_transformed, hyp_transformed)
    })

results_df = pd.DataFrame(results)
print(results_df)

          key                          ref_file             hyp_file  \
0   Mf110_NAp  Mf110_NAp_transkrypcja_clean.txt  Mf110_NAp_clean.txt   
1   Bm020_NAp  Bm020_NAp_transkrypcja_clean.txt  Bm020_NAp_clean.txt   
2   Bf068_NAp  Bf068_NAp_transkrypcja_clean.txt  Bf068_NAp_clean.txt   
3   Bm015_NAp  Bm015_NAp_transkrypcja_clean.txt  Bm015_NAp_clean.txt   
4   Mm800_NAp  Mm800_NAp_transkrypcja_clean.txt  Mm800_NAp_clean.txt   
..        ...                               ...                  ...   
88  Bm010_NAp  Bm010_NAp_transkrypcja_clean.txt  Bm010_NAp_clean.txt   
89  Mf119_NAp  Mf119_NAp_transkrypcja_clean.txt  Mf119_NAp_clean.txt   
90  Bm025_NAp  Bm025_NAp_transkrypcja_clean.txt  Bm025_NAp_clean.txt   
91  Mf115_NAp  Mf115_NAp_transkrypcja_clean.txt  Mf115_NAp_clean.txt   
92  Mm008_NAp  Mm008_NAp_transkrypcja_clean.txt  Mm008_NAp_clean.txt   

         WER       CER       WIL       MER       WIP  
0   0.133479  0.091203  0.190311  0.131324  0.809689  
1   0.222675  0.154843  0

In [47]:
# calculating mean metrics

mean_metrics = results_df[['WER', 'CER', 'WIL', 'MER', 'WIP']].mean().to_frame(name='Mean').reset_index()
mean_metrics.columns = ['Metric', 'Mean']
print(mean_metrics)
    

  Metric      Mean
0    WER  0.307929
1    CER  0.242144
2    WIL  0.360338
3    MER  0.283971
4    WIP  0.639662


In [ ]:
# audio quality metrics

audio_metric_df = get_audio_metrics_dataframe(used_audio_path)
final_df = pd.merge(results_df, audio_metric_df, left_on="key", right_on="key")

output_name = f"ASR_results_{mode}.csv"
final_df.to_csv(output_name, index=False)
print("Saved:", output_name)


Processing audio files:  54%|█████▍    | 139/256 [03:35<03:03,  1.57s/it]